This notebook preprocesses the cell towers dataset into an efficient set of parquet files. First register for an API key with [opencellid](https://www.opencellid.org/) and then download the `cell_towers.csv.gz` file.  Unzip it and modify the `cell_towers_path` below to the path to your `cell_towers.csv` file.  Then execute this notebook to write out a compressed parquet to the `../data` directory.

In [15]:
#!pip install dask
#!pip install pyproj
#!pip install -r ../requirements.txt
#!pip3 install pandas
#!pip install pyarrow

You should consider upgrading via the 'c:\users\dethe\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
cell_towers_path = "cell_towers.csv"

In [16]:
# Imports
import dask.dataframe as dd
import pandas as pd
import pyproj
from pyproj import Transformer
import requests
import os
import pyarrow

In [4]:
# From https://www.opencellid.org/downloads.php
# Field Descriptions: http://wiki.opencellid.org/wiki/Menu_map_view#Cells_database:
ddf = dd.read_csv(cell_towers_path)
ddf.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal
0,UMTS,262,2,801,86355,0,13.285512,52.522202,1000,7,1,1282569574,1300155341,0
1,GSM,262,2,801,1795,0,13.276907,52.525714,5716,9,1,1282569574,1300155341,0
2,GSM,262,2,801,1794,0,13.285064,52.524000,6280,13,1,1282569574,1300796207,0
3,UMTS,262,2,801,211250,0,13.285446,52.521744,1000,3,1,1282569574,1299466955,0
4,UMTS,262,2,801,86353,0,13.293457,52.521515,1000,2,1,1282569574,1291380444,0


In [5]:
ddf['radio'] = ddf.radio.astype('category')

ddf['created'] = dd.to_datetime(ddf.created, unit='s')
ddf['created'].dt.tz_localize('CET', ambiguous='infer')

ddf['updated'] = dd.to_datetime(ddf.created, unit='s')
ddf['updated'].dt.tz_localize('CET', ambiguous='infer')

ddf = ddf[dd.to_datetime(ddf.created) >= '2003']

In [6]:
# Categorize radio
ddf['radio'] = ddf.radio.astype('category')

# Created and updated to datetime integers
#ddf['created'] = dd.to_datetime(ddf.created, unit='s').astype('int')   #note -  .astype is deprecated
#ddf['updated'] = dd.to_datetime(ddf.updated, unit='s').astype('int')   #note - .astype is deprecated
ddf['created'] = dd.to_datetime(ddf.created, unit='s')
ddf['created'].dt.tz_localize('CET', ambiguous='infer')

ddf['updated'] = dd.to_datetime(ddf.created, unit='s')
ddf['updated'].dt.tz_localize('CET', ambiguous='infer')

# Filter out outliers created before 2003
ddf = ddf[dd.to_datetime(ddf.created) >= '2003']

In [7]:
# convert lon/lat to epsg:3857 (psuedo-mercator) so generated images
# can be overlayed on a Mercator projected map
transformer = Transformer.from_crs("epsg:4326", "epsg:3857")
def to3857(df):
    x_3857, y_3857 = transformer.transform(df.lat.values, df.lon.values)
    return df.assign(x_3857=x_3857, y_3857=y_3857)

ddf = ddf.map_partitions(to3857)

ddf.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,x_3857,y_3857
0,UMTS,262,2,801,86355,0,13.285512,52.522202,1000,7,1,2010-08-23 13:19:34,2010-08-23 13:19:34,0,1.478936e+06,6.895103e+06
1,GSM,262,2,801,1795,0,13.276907,52.525714,5716,9,1,2010-08-23 13:19:34,2010-08-23 13:19:34,0,1.477979e+06,6.895745e+06
2,GSM,262,2,801,1794,0,13.285064,52.524000,6280,13,1,2010-08-23 13:19:34,2010-08-23 13:19:34,0,1.478887e+06,6.895432e+06
3,UMTS,262,2,801,211250,0,13.285446,52.521744,1000,3,1,2010-08-23 13:19:34,2010-08-23 13:19:34,0,1.478929e+06,6.895019e+06
4,UMTS,262,2,801,86353,0,13.293457,52.521515,1000,2,1,2010-08-23 13:19:34,2010-08-23 13:19:34,0,1.479821e+06,6.894977e+06


In [8]:
#!pip3 install lxml
#!pip3 install html5lib


In [11]:
import html5lib
import lxml as lxml

# Download network info for mcc/mnc from 'https://cellidfinder.com/mcc-mnc'
html = requests.get('https://cellidfinder.com/mcc-mnc')
tables = pd.read_html(html.content)
mcc_mnc_df = pd.concat(tables).reset_index(drop=True)

# Create description column as Network, falling back to "Operator or branch" if Network not found
mcc_mnc_df['Description'] = mcc_mnc_df.Network.where(
    ~pd.isnull(mcc_mnc_df.Network), mcc_mnc_df['Operator or brand name']
)

# Drop unneeded columns
codes = mcc_mnc_df.drop(['Network', 'Operator or brand name'], axis=1)
codes.head()

,MCC,MNC,Status,Description
0,289,67,Operational,Aquafon
1,289,68,Operational,A-Mobile
2,412,1,Operational,Afghan Wireless Communication Company
3,412,20,Operational,Telecom Development Company Afghanistan Ltd.
4,412,40,Operational,MNT Group Afganistan


In [12]:
# Categorize non-numeric columns
for col, dtype in codes.dtypes.items():
    if dtype == 'object':
        codes[col] = codes[col].astype('category')

In [13]:
# Merge mnc/mcc info with cell towers dataset
ddf_merged = ddf.merge(codes, left_on=['mcc', 'net'], right_on=['MCC', 'MNC'], how='left')
ddf_merged

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,x_3857,y_3857,MCC,MNC,Status,Description
npartitions=112,,,,,,,,,,,,,,,,,,,,
,category[unknown],int64,int64,int64,int64,int64,float64,float64,int64,int64,int64,datetime64[ns],datetime64[ns],int64,float64,float64,int64,int64,category[known],category[known]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [17]:
# Write parquet file to ../data directory
os.makedirs('../data', exist_ok=True)
parquet_path = '../data/cell_towers.parq'
ddf_merged.to_parquet(parquet_path, compression='snappy')

In [18]:
# Read and display the first three rows (Transpose so can see all of the columns)
dd.read_parquet(parquet_path).head(3).T

,0,1,2
radio,UMTS,GSM,GSM
mcc,262,262,262
net,2,2,2
area,801,801,801
cell,86355,1795,1794
unit,0,0,0
lon,13.285512,13.276907,13.285064
lat,52.522202,52.525714,52.524
range,1000,5716,6280
samples,7,9,13
